In [20]:
# GPU details
!nvidia-smi

Sun Dec 29 05:13:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    33W / 250W |  15765MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# CPU details
!cat /proc/cpuinfo

In [0]:
# suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [0]:
import os # for basic operations like folder creation, directory validation etc
import glob # finds file according to wildcard given
import shutil # for moving files
import matplotlib.pyplot as plt # for plotting graphs and viewing images

In [25]:
'''
-> Generator class for applying image transformations
-> Can be used for image augmentation purposes if need be
-> Currently just normalises the image
-> tensorflow does not support importing and processing images
    so use this or use PIL(python image library)
'''

from keras.models import Model
from keras import backend as K
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
from PIL import ImageFile

# ImageFile.LOAD_TRUNCATED_IMAGES = True

# Image dimension data
'''
    Sensor Type	    Image Size	            Set A	Set B	Resolution
DB1	Optical Sensor	640x480 (307 Kpixels)	100x8	10x8	500 dpi
DB2	Optical Sensor	328x364 (119 Kpixels)	100x8	10x8	500 dpi
DB3	Thermal sweeping
    Sensor	        300x480 (144 Kpixels)	100x8	10x8	512 dpi
DB4	SFinGe v3.0	    288x384 (108 Kpixels)	100x8	10x8	about 500 dpi
'''

img_width, img_height = 640, 480
train_data_dir = './drive/My Drive/dbs/DB1_B/dataset'
validation_data_dir = '../input/fingerprint-db/DB/DB_1/dataset'
nb_train_samples = 1200
nb_validation_samples = 300
batch_size = 16
epochs = 100

if K.image_data_format() == 'channels_last':
    input_tensor = Input(shape=(img_width, img_height, 3))
else:
    input_tensor = Input(shape=(3, img_width, img_height))


train_datagen = ImageDataGenerator(
    rescale=1./255,
#     horizontal_flip=True,
    rotation_range=30,
#     zoom_range=[0.1, 0.4],
    validation_split=0.40,
)

# test_datagen = ImageDataGenerator(
#     rescale=1./255,
#     horizontal_flip=True,
#     rotation_range=30,
#     zoom_range=[0.1, 0.4],
# )

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

# imports the inceptionv3 pretained model
inception_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
# x = Dropout(0.20)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.20)(x)
# x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inception_model.input, outputs=predictions)

# freeze all convolutional InceptionV3 layers
# for layer in inception_model.layers:
#     layer.trainable = False

# functions to calculate f1score
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy', f1score, recall_m, precision_m])
# model.summary()

filepath_accuracy = 'accuracy_weights.{epoch:02d}-{val_acc:.2f}.hdf5'
filepath_f1score = 'f1score_weights.{epoch:02d}-{val_f1score:.2f}.hdf5'

csv_logger = CSVLogger('training.log')

accuracy_checkpoint = ModelCheckpoint(filepath_accuracy, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
f1score_checkpoint = ModelCheckpoint(filepath_f1score, monitor='val_f1score', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        shuffle=True,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[csv_logger, accuracy_checkpoint, f1score_checkpoint]
        )

Found 50 images belonging to 10 classes.
Found 30 images belonging to 10 classes.
Epoch 1/100
75/75 [==============================] - 107s 1s/step - loss: 2.4520 - acc: 0.1275 - f1score: 0.0104 - recall_m: 0.0067 - precision_m: 0.0694 - val_loss: 8.8193 - val_acc: 0.1333 - val_f1score: 0.1070 - val_recall_m: 0.1000 - val_precision_m: 0.1156
Epoch 2/100
75/75 [==============================] - 85s 1s/step - loss: 2.1432 - acc: 0.1526 - f1score: 0.0126 - recall_m: 0.0067 - precision_m: 0.1070 - val_loss: 13.4665 - val_acc: 0.1111 - val_f1score: 0.1019 - val_recall_m: 0.1000 - val_precision_m: 0.1039
Epoch 3/100
75/75 [==============================] - 86s 1s/step - loss: 2.0333 - acc: 0.1703 - f1score: 0.0165 - recall_m: 0.0092 - precision_m: 0.0892 - val_loss: 8.9833 - val_acc: 0.1037 - val_f1score: 0.1347 - val_recall_m: 0.0926 - val_precision_m: 0.2526
Epoch 4/100
75/75 [==============================] - 87s 1s/step - loss: 1.9968 - acc: 0.1857 - f1score: 0.0000e+00 - recall_m: 0.000

In [0]:
model.save('inceptionv3.h5')

In [26]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for recall
plt.plot(history.history['recall_m'])
plt.plot(history.history['val_recall_m'])
plt.title('model recall_m')
plt.ylabel('recall_m')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for precision
plt.plot(history.history['precision_m'])
plt.plot(history.history['val_precision_m'])
plt.title('model precision_m')
plt.ylabel('precision_m')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for f1score
plt.plot(history.history['f1score'])
plt.plot(history.history['val_f1score'])
plt.title('model f1score')
plt.ylabel('f1score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['val_loss', 'val_acc', 'val_f1score', 'val_recall_m', 'val_precision_m', 'loss', 'acc', 'f1score', 'recall_m', 'precision_m'])


KeyError: ignored